<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/DAX30_Index_Forecasting_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import necessary libraries
!pip install --upgrade tensorflow
import datetime
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler

# Function to calculate RSI
def calculate_rsi(data, window=14):
    delta = data['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    return 100 - (100 / (1 + RS))

# Function to calculate MACD
def calculate_macd(data, slow=26, fast=12, signal=9):
    exp1 = data['Close'].ewm(span=fast, adjust=False).mean()
    exp2 = data['Close'].ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

# Download data for GER30 (DAX)
symbol = "^GDAXI"
start_date = "2010-01-01"
end_date = datetime.datetime.now().strftime("%Y-%m-%d")
df = yf.download(symbol, start=start_date, end=end_date, interval="1d")

# Calculate RSI and MACD
df['RSI'] = calculate_rsi(df)
macd, signal_line = calculate_macd(df)
df['MACD'] = macd
df['MACD_Signal'] = signal_line

# Drop NaN values
df.dropna(inplace=True)

# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df[['Close', 'RSI', 'MACD', 'MACD_Signal']])

# Define a function to create sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length), :]
        y = data[i+seq_length, 0]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Prepare training and test datasets
seq_length = 60
X, y = create_sequences(df_scaled, seq_length)
train_size = int(len(df) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build LSTM Model with dropout
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(seq_length, 4)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=32, epochs=10)

# [Prediction and advice code remains the same]


# Preprocess data: Normalize 'Close' prices
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df['Close'].values.reshape(-1,1))

# Define a function to create sequences
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Prepare training and test datasets
seq_length = 60
X, y = create_sequences(df_scaled, seq_length)
train_size = int(len(df) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build LSTM Model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=32, epochs=10)

# Predict the trends for the next 5 days
predicted_prices = []
current_test_sequence = X_test[-1]  # Start with the last sequence in test data
for _ in range(5):
    current_test_sequence = current_test_sequence.reshape((1, seq_length, 1))
    predicted_price = model.predict(current_test_sequence)
    predicted_prices.append(predicted_price[0][0])
    # Update the sequence for the next prediction
    predicted_price = predicted_price.reshape((1, 1, 1))  # Reshape predicted_price
    current_test_sequence = np.append(current_test_sequence[:, 1:, :], predicted_price, axis=1)

# Inverse transform to get actual price predictions
predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

# Provide Buy, Sell, or Hold advice based on predictions
for i, price in enumerate(predicted_prices):
    action = "Hold"
    if i > 0 and price > predicted_prices[i-1]:
        action = "Buy"
    elif i > 0 and price < predicted_prices[i-1]:
        action = "Sell"
    print(f"Day {i+1}: Predicted Price: {price[0]}, Action: {action}")

##############from datetime import datetime, timedelta
from datetime import datetime, timedelta
from IPython.display import display, HTML

# Assuming today's date for starting the prediction
last_date = datetime.now()

# Inverse transform to get actual price predictions
predicted_prices = scaler.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

# Calculating growth rate for each day of the next 5 days
growth_rate_each_day = []

# Getting the last known close price (most recent trading day)
today_close = df['Close'].iloc[-1]

# Generate the dates for the next 5 days (excluding weekends)
dates = []
for i in range(1, 10):  # Check the next 10 days, as some might be weekends
    next_day = last_date + timedelta(days=i)
    # Check if it's a weekday (0=Monday, 1=Tuesday, ..., 6=Sunday)
    if next_day.weekday() < 5:
        dates.append(next_day.strftime('%m/%d/%Y'))
    if len(dates) == 5:
        break

for i, predicted_price in enumerate(predicted_prices):
    rate = (predicted_price - today_close) / today_close
    growth_rate_each_day.append((dates[i], rate[0]))

# Function to colorize text based on value
def colorize_text(value):
    color = "green" if value >= 0 else "red"
    return f'<span style="color: {color}">{value*100:.2f}%</span>'

# Displaying growth rate for each day in a colorized format
output = "Growth rates for the next 5 days:<br>"
for date, rate in growth_rate_each_day:
    output += f"{date}: {colorize_text(rate)}<br>"
display(HTML(output))


[*********************100%%**********************]  1 of 1 completed
Epoch 1/10
89/89 [==============================] - 9s 53ms/step - loss: 0.0176
Epoch 2/10
89/89 [==============================] - 4s 48ms/step - loss: 0.0030
Epoch 3/10
89/89 [==============================] - 6s 65ms/step - loss: 0.0025
Epoch 4/10
89/89 [==============================] - 4s 48ms/step - loss: 0.0021
Epoch 5/10
89/89 [==============================] - 4s 48ms/step - loss: 0.0019
Epoch 6/10
89/89 [==============================] - 6s 64ms/step - loss: 0.0017
Epoch 7/10
89/89 [==============================] - 4s 48ms/step - loss: 0.0017
Epoch 8/10
89/89 [==============================] - 4s 48ms/step - loss: 0.0014
Epoch 9/10
89/89 [==============================] - 6s 64ms/step - loss: 0.0014
Epoch 10/10
89/89 [==============================] - 4s 48ms/step - loss: 0.0012
Epoch 1/10
89/89 [==============================] - 10s 64ms/step - loss: 0.0182
Epoch 2/10
89/89 [==============================]